In [244]:
import os
import time
import scipy.io as sio
import numpy as np
from skimage.measure import regionprops

from my_funcs.plot_functions import t1_t2_pixel_reader

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.mask_functions import mask_processor, tumor_masks_processor, tumor_contra_masks_processor, contour_finder

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from sequential_nn.model import Network
from utils.normalization import normalize_range, un_normalize_range

In [244]:
import os
import time
import scipy.io as sio
import numpy as np
from skimage.measure import regionprops

from my_funcs.plot_functions import t1_t2_pixel_reader

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.mask_functions import mask_processor, tumor_masks_processor, tumor_contra_masks_processor, contour_finder

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from sequential_nn.model import Network
from utils.normalization import normalize_range, un_normalize_range

In [244]:
import os
import time
import scipy.io as sio
import numpy as np
from skimage.measure import regionprops

from my_funcs.plot_functions import t1_t2_pixel_reader

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.mask_functions import mask_processor, tumor_masks_processor, tumor_contra_masks_processor, contour_finder

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from sequential_nn.model import Network
from utils.normalization import normalize_range, un_normalize_range

# Subject choice (0/1/2) #

In [245]:
# subject_i = 1  # 0, 1, 2, 3
save_name = 'mouse_july_adult_tumor'

## subject data ##

In [246]:
from my_funcs.mouse_data_dicts import subject_dicts_july_adult_tumor_25

# Root stats:
general_fn = os.path.abspath(os.curdir)
current_dir = os.getcwd()  # Get the current directory
parent_dir = os.path.dirname(os.path.dirname(current_dir))  # Navigate up one directory level

# Subject data:
txt_file_name = 'labarchive_notes.txt'

In [247]:
fig = make_subplots(rows=3, cols=4, horizontal_spacing=0.01, vertical_spacing=0.08, 
                    subplot_titles=['t1 [ms]', 't2 [ms]', 't2_highres', 'Wasser'])

# for subject_i, subject_dict in enumerate(subject_dicts):
for subject_i, subject_dict in enumerate(subject_dicts_july_adult_tumor_25):
    row_i = subject_i+1
    # sub_i = subject_dicts[subject_i]
    phantom_choice = subject_i+1
    subject_dict['save_name'] = save_name
    glu_mouse_fn = os.path.join(parent_dir, 'data', 'scans', subject_dict['scan_name'], subject_dict['sub_name'])
    
    # import nrrd
    # mask_fn = os.path.join(glu_mouse_fn, 'slicer_masks', 'mask.seg.nrrd')
    # mask = nrrd.read(mask_fn, index_order='C')[0][2, : ,:]
    # print(mask.argmax())
    # print(subject_dict['mask_slice'])
    mask = mask_processor(glu_mouse_fn, subject_dict['resratio'], idx=subject_dict['mask_slice'])

    # crop image
    r_c, c_c = regionprops(mask)[0].centroid
    r_c, c_c = int(r_c), int(c_c)
    bb_r_h = 18
    bb_c_w = 25

    if os.path.exists(os.path.join(glu_mouse_fn, 'slicer_masks', 'tumor.seg.nrrd')):
        tumor_mask, mirror_mask = tumor_masks_processor(glu_mouse_fn, subject_dict['resratio'], subject_dict['mask_slice'])
        tumor_mask = tumor_mask[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
        mirror_mask = mirror_mask[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]

        tumor_cntr = contour_finder(tumor_mask)  # not accurate needs fixing!
        if np.any(mirror_mask):
            mirror_cntr = contour_finder(mirror_mask)

    mask = mask[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]

    glu_mouse_fn = os.path.join(parent_dir, 'data', 'scans', subject_dict['scan_name'], subject_dict['sub_name'])
    highres_fn, _, bruker_dataset_highres = bruker_dataset_creator(glu_mouse_fn, txt_file_name, 'T2_MSME_high')
    wassr_fn, _, bruker_dataset_wassr = bruker_dataset_creator(glu_mouse_fn, txt_file_name, 'WASSR')

    # full_mask = subject_dict['roi_mask']
    date = subject_dict['month']
    save_name = subject_dict['save_name']
    sub_name = subject_dict['sub_name']
    temp = subject_dict['temp']
    highres_img_idx = subject_dict['highres_img_idx']
    resratio = subject_dict['resratio']

    # highres_img_idx = 4
    t1 = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1',
                                   image_file=4)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
    t2 = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
    t2_highres = dicom_data_arranger(bruker_dataset_highres, highres_fn).astype(np.float)[highres_img_idx,(r_c-bb_r_h)*resratio:(r_c+bb_r_h)*resratio, (c_c-bb_c_w)*resratio:(c_c+bb_c_w)*resratio]
    wassr = dicom_data_arranger(bruker_dataset_wassr, wassr_fn).astype(np.float)[0,r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]

    # Loading the separately acquired water_t1t2-maps
    shift_up, shift_right = subject_dict['t_shift']
    t1 = np.roll(t1, shift=(shift_up, shift_right), axis=(0, 1))
    t2 = np.roll(t2, shift=(shift_up, shift_right), axis=(0, 1))
    t2_highres = np.roll(t2_highres, shift=(shift_up*resratio, shift_right*resratio), axis=(0, 1))

    # Add heatmaps for the three arrays
    heatmap_t1 = go.Heatmap(z=t1*mask, showscale=False, zmin=700, zmax=2400) # 500-2400?
    heatmap_t2 = go.Heatmap(z=t2*mask,showscale=False, zmin=35, zmax=160) # 35-200?
    heatmap_t2_highres = go.Heatmap(z=t2_highres, showscale=False)
    heatmap_wassr = go.Heatmap(z=wassr, showscale=False)


    fig.add_trace(heatmap_t1, row=row_i, col=1)
    fig.add_trace(heatmap_t2, row=row_i, col=2)
    fig.add_trace(heatmap_t2_highres, row=row_i, col=3)
    fig.add_trace(heatmap_wassr, row=row_i, col=4)
    
    fig.update_yaxes(title_text=sub_name, showgrid=False, row=row_i, col=1)

fig.update_layout(
    template='plotly_dark',  # Set the theme to plotly dark
    title_text=f"{sub_name} {date} {temp}°C",
    showlegend=False,  # Hide legend
    height=450,
    width=1000,  # Set a width based on your preference
    margin=dict(l=5, r=40, t=60, b=20),  # Adjust top and bottom margins
    title=dict(x=0.02, y=0.97)  # Adjust the title position
)

# Add individual titles and separate colorbars
for j in range(3):
    for i, title in enumerate([1, 2, 3, 4], start=1):
        fig.update_xaxes(row=j+1, col=i, showgrid=False, showticklabels=False)
        fig.update_yaxes(showgrid=False, row=j+1, col=i, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# fig.update_yaxes(title_text='july 8', showgrid=False, row=1, col=1)
# fig.update_yaxes(title_text='july 15', showgrid=False, row=2, col=1)
# fig.update_yaxes(title_text='july 29', showgrid=False, row=2, col=1)
# Show the plot
fig.show()

In [248]:
from utils.colormaps import custom_hot, custom_aggrnyl, custom_viridis

# pH image #

In [249]:
from my_funcs.cest_functions import z_spec_rearranger, offset_rearranger, wassr_b0_mapping, b0_correction

# z-spec for roi
def correct_b0(subject_dict, glu_mouse_fn, txt_file_name, mask):
    # given:
    gyro_ratio_hz = 42.5764  # for H [Hz/uT]
    b0 = 7
    
    b1_names = subject_dict['z_b1s_names']
    
    # WASSR image
    wassr_dicom_fn, wassr_mrf_files_fn, wassr_bruker_dataset = bruker_dataset_creator(glu_mouse_fn, txt_file_name,
                                                                                      'WASSR')
    wassr_data = dicom_data_arranger(wassr_bruker_dataset, wassr_dicom_fn)
    M0_wassr, arr_wassr_spec = z_spec_rearranger(wassr_data)  # (21, 64, 64)
    
    wassr_norm = np.divide(arr_wassr_spec, np.where(M0_wassr == 0, 1e-8, M0_wassr))  # (22, 64, 64) full_mask
    offset_hz = offset_rearranger(wassr_bruker_dataset['SatFreqList'].value)
    offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    b0_map = wassr_b0_mapping(wassr_norm, mask, w_x=offset_ppm, MainFieldMHz=gyro_ratio_hz * b0)
    
    z_dict = {}
    for b1_i, b1_name in enumerate(b1_names):  # 0.7, 1.5, 2, 4, 6
        b1_dict = {}
        # z-spec
        glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_mouse_fn,
                                                                                                txt_file_name,
                                                                                                b1_name)  # (72/58/52, 64, 64) [M0,7,-7,6.75,-6.75,...,0.25,-0.25,0]
        cest_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
        M0_cest, arr_z_spec = z_spec_rearranger(cest_data)
        z_spec_norm = np.divide(arr_z_spec, np.where(M0_cest == 0, 1e-8, M0_cest))  # (51/57, 64, 64) full_mask
    
        # offset vector
        offset_hz = offset_rearranger(bruker_dataset['SatFreqList'].value)
        offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    
        b0_cor_zspec = b0_correction(b0_map, z_spec_norm, offset_hz)  # have not checked!
        
        fr_s = int(bruker_dataset['SatFreqStart'].value)
        fr_e = int(bruker_dataset['SatFreqEnd'].value)
        fr_i = round(bruker_dataset['SatFreqInc'].value, 2)
    
        ppm = np.arange(fr_s, fr_e-fr_i, -fr_i)
    
        b1_dict['ppm'] = ppm
        b1_dict['M0'] = M0_cest
        b1_dict[f'z_b1cor'] = b0_cor_zspec
        b1_dict[f'z_nob1cor'] = z_spec_norm
        z_dict[f'{b1_name}'] = b1_dict
        
    return z_dict

In [250]:
from lmfit import Model

# Sample data (replace this with your actual data)
x = np.array([5.5, 6, 6.5, 7, 7.5])
y = np.array([1923, 2019, 3489, 7248, 14143])

# Define a polynomial model (degree 2)
def polynomial(x, a, b, c):
    return a * x**2 + b * x + c

# Create a model from the polynomial function
model = Model(polynomial)

# Make an initial guess of the parameters
params = model.make_params(a=1, b=1, c=1)

# Fit the model to the data
result = model.fit(y, params, x=x)

# # Print the fit report
# print(result.fit_report())
# 
# # Predict using the fitted model
# x_new = np.linspace(min(x), max(x), 100)
# y_new = result.eval(x=x_new)
# 
# # Plot the original data and the fitted polynomial curve
# plt.scatter(x, y, color='blue', label='Data')
# plt.plot(x_new, y_new, color='red', linestyle='--', label='Polynomial Fit')
# plt.xlabel('pH')
# plt.ylabel('Exchange Rate')
# plt.title('pH vs Exchange Rate (Polynomial Fit)')
# plt.legend()
# plt.tight_layout()  # Adjust subplot parameters to give specified padding
# plt.savefig(f'{subject_nn_image_path}/ph_fit.jpeg')
# plt.show()

In [251]:
def aacid_roi(subject_dict, z_dict, roi_mask):
    # Iterate over each channel
    for b1_i, b1_name in enumerate(subject_dict['z_b1s_names']):
        # Iterate over each image
        ppm = z_dict[b1_name]['ppm']
        M0 = z_dict[b1_name]['M0']
        z_b1cor = z_dict[b1_name]['z_b1cor']
        z_nob1cor = z_dict[b1_name]['z_nob1cor']
    
        cha_n, r_n, c_n = z_b1cor.shape
    
        if b1_name == '1p5uT':
            MainFieldMHz = 298
            B1_uT = 1.5
            gyro_ratio_hz = 42.5764
            w1_Hz = round(B1_uT*gyro_ratio_hz)  # satpwr for scan (Hz)
            w_Hz = ppm * MainFieldMHz  # Scanned frequency offsets (ppm * MHz = Hz)
    
            # Calculate mean value for each channel in the masked regions (excluding zeros)
            roi_mask = roi_mask.astype(bool)
            expanded_roi_mask = np.repeat(roi_mask[np.newaxis, :, :], z_b1cor.shape[0], axis=0)

            masked_z_b1cor = np.ma.masked_where(expanded_roi_mask == 0, z_b1cor)
            roi_mean_z = np.mean(masked_z_b1cor, axis=(1,2))
            Interp_w_Hz = np.arange(min(w_Hz), max(w_Hz)+1, 1)  # Spline interpolation of scanned offsets every 1 Hz
            tck = splrep(w_Hz[::-1], roi_mean_z[::-1], s=0)  # Spline representation of the data (I needed to reverse)
            M_2p75 = splev(2.75*MainFieldMHz, tck)
            M_3p5 = splev(3.5*MainFieldMHz, tck)
    
            M_6 = roi_mean_z[np.argmin(abs(ppm-6))]
            aacid_roi = (M_3p5 * (M_6 - M_2p75)) / (M_2p75 * (M_6 - M_3p5))
    
            ph_roi = (-4*aacid_roi + 12.8)  # in-vivo calibration
            # ph_roi = (aacid_roi - 6.5) / (-0.64)  # in-vitro calibration
            # ph_roi = (aacid_roi - 1.79) / (-0.236)  # in-vitro egg calibration physiological (not accurate!!!)
            # ph_roi = (aacid_roi - 1.95) / (-0.176)  # in-vitro egg calibration all (not accurate!!!)
            
    return aacid_roi, ph_roi, roi_mean_z

In [252]:
from scipy.interpolate import splev, splrep
import pandas as pd

# Create subplots with 1 row and 3 columns, increased horizontal spacing
fig = make_subplots(rows=3, cols=5, horizontal_spacing=0.004, vertical_spacing=0.004, 
                    subplot_titles=['t2 highres', 't1 [ms]', 't2 [ms]', 'AACID map', 'pH map'])
z_fig = make_subplots(rows=3, cols=1, horizontal_spacing=0.004, vertical_spacing=0.004)

aacid_dfs = []
ph_dfs = []
flattened_aacid_dicts = []
flattened_ph_dicts = []
# for subject_i, subject_dict in enumerate(subject_dicts):
for subject_i, subject_dict in enumerate(subject_dicts_july_adult_tumor_25):
    row_i = subject_i+1
    phantom_choice = subject_i+1
    subject_dict['save_name'] = save_name
    highres_img_idx = subject_dict['highres_img_idx']
    resratio = subject_dict['resratio']
    glu_mouse_fn = os.path.join(parent_dir, 'data', 'scans', subject_dict['scan_name'], subject_dict['sub_name'])

    mask = mask_processor(glu_mouse_fn, subject_dict['resratio'], subject_dict['mask_slice'])
    z_dict = correct_b0(subject_dict, glu_mouse_fn, txt_file_name, mask)

    # Iterate over each channel
    for b1_i, b1_name in enumerate(subject_dict['z_b1s_names']):
        # Iterate over each image
        ppm = z_dict[b1_name]['ppm']
        z_b1cor = z_dict[b1_name]['z_b1cor']
        z_nob1cor = z_dict[b1_name]['z_nob1cor']

        cha_n, r_n, c_n = z_b1cor.shape

        aacid_map = np.zeros([r_n, c_n])
        if b1_name == '1p5uT':
            MainFieldMHz = 298
            B1_uT = 1.5
            gyro_ratio_hz = 42.5764
            w1_Hz = round(B1_uT*gyro_ratio_hz)  # satpwr for scan (Hz)

            w_Hz = ppm * MainFieldMHz  # Scanned frequency offsets (ppm * MHz = Hz)

            Interp_w_Hz = np.arange(min(w_Hz), max(w_Hz)+1, 1)  # Spline interpolation of scanned offsets every 1 Hz
            for r_i in range(r_n):
                for c_i in range(c_n):
                    if mask[r_i, c_i] != 0:
                        cur_z = z_b1cor[:, r_i, c_i]
                        tck = splrep(w_Hz[::-1], z_b1cor[::-1, r_i, c_i], s=0)  # Spline representation of the data (I needed to reverse)
                        M_2p75 = splev(2.75*MainFieldMHz, tck)
                        M_3p5 = splev(3.5*MainFieldMHz, tck)
                        # M_2 = z_b1cor[np.argmin(abs(ppm-2)), r_i, c_i]
                        M_6 = z_b1cor[np.argmin(abs(ppm-6)), r_i, c_i]
                        
                        aacid_map[r_i, c_i] = (M_3p5 * (M_6 - M_2p75)) / (M_2p75 * (M_6 - M_3p5))
                        # aacid_map[r_i, c_i] = (M_3p5 * (M_6 - M_2)) / (M_2 * (M_6 - M_3p5))
                        
                        # ph = (AACID - 6.5) / (-0.64)
                        
                        # # Add scatter plot z-spectrum element
                        # z_fig.add_trace(go.Scatter(
                        #     x=ppm,
                        #     y=cur_z,
                        #     mode='lines',
                        #     line=dict(color='blue', width=2),
                        #     opacity=0.5
                        # ), row=row_i, col=1)
            # ph_map = (aacid_map - 6.1357681)/-0.6265661  # my bsa calibration
            # ph_map = (aacid_map - 6.5) / (-0.64)  # their bsa calibration
            # AACID = -0.6265661 * ph + 6.1357681

            ph_map = (-4*aacid_map + 12.8)  # in vivo calibration
            # ph_map = (aacid_map - 2.8357681) / -0.2365661  # projection trick
            ph_map[ph_map < 0] = 0
            ph_map[ph_map > 10] = 11
            ph_map = ph_map * mask
            f_aacid_map = aacid_map
    
    predicted_ksw = result.eval(x=ph_map) * mask
    masked_ph = np.ma.masked_where(mask == 0, ph_map)
    masked_aacid = np.ma.masked_where(mask == 0, f_aacid_map)
    whole_brain_roi_ph = f'{np.mean(masked_ph):.2f} ± {np.std(masked_ph):.2f}'
    whole_brain_roi_aacid = f'{np.mean(masked_aacid):.2f} ± {np.std(masked_aacid):.2f}'

    mask = mask_processor(glu_mouse_fn, subject_dict['resratio'], subject_dict['mask_slice'])

    # crop image
    r_c, c_c = regionprops(mask)[0].centroid
    r_c, c_c = int(r_c), int(c_c)
    bb_r_h = 17
    bb_c_w = 25

    if os.path.exists(os.path.join(glu_mouse_fn, 'slicer_masks', 'tumor.seg.nrrd')):
        # if row_i==3:
        #     tumor_mask, mirror_mask = tumor_masks_processor(glu_mouse_fn, subject_dict['resratio'], subject_dict['mask_slice'])
        # else:
        tumor_mask, mirror_mask = tumor_contra_masks_processor(glu_mouse_fn, subject_dict['resratio'], subject_dict['mask_slice'])
        
        # tumor ph
        tumor_masked_ph = np.ma.masked_where(tumor_mask == 0, ph_map)
        tumor_roi_mean_pixels_ph = f'{np.mean(tumor_masked_ph):.2f} ± {np.std(tumor_masked_ph):.2f}'
        tumor_roi_mean_z_ph = f'{aacid_roi(subject_dict, z_dict, tumor_mask)[1]:.2f}'
        # Add scatter plot z-spectrum element
        _, _, roi_mean_z = aacid_roi(subject_dict, z_dict, tumor_mask)
        z_fig.add_trace(go.Scatter(
            x=ppm,
            y=roi_mean_z,
            mode='lines',
            line=dict(color='red', width=2),
            opacity=1
        ), row=row_i, col=1)
    
        # tumor aacid
        tumor_masked_aacid = np.ma.masked_where(tumor_mask == 0, f_aacid_map)
        tumor_roi_mean_pixels_aacid = f'{np.mean(tumor_masked_aacid):.2f} ± {np.std(tumor_masked_aacid):.2f}'
        tumor_roi_mean_z_aacid = f'{aacid_roi(subject_dict, z_dict, tumor_mask)[0]:.2f}'
        
        # tumor final masks
        tumor_mask_cropped_ph = (tumor_mask*ph_map)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
        tumor_mask_cropped_aacid = (tumor_mask*f_aacid_map)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
        tumor_cntr = contour_finder(tumor_mask_cropped_ph)  # not accurate needs fixing!
        
        # contra ph
        mirror_masked_ph = np.ma.masked_where(mirror_mask == 0, ph_map)
        contra_roi_mean_pixels_ph = f'{np.mean(mirror_masked_ph):.2f} ± {np.std(mirror_masked_ph):.2f}'
        contra_roi_mean_z_ph= f'{aacid_roi(subject_dict, z_dict, mirror_mask)[1]:.2f}'
        # Add scatter plot z-spectrum element
        _, _, roi_mean_z = aacid_roi(subject_dict, z_dict, mirror_mask)
        z_fig.add_trace(go.Scatter(
            x=ppm,
            y=roi_mean_z,
            mode='lines',
            line=dict(color='white', width=2),
            opacity=1
        ), row=row_i, col=1)
        
        # contra aacid
        contra_roi_mean_z_aacid = f'{aacid_roi(subject_dict, z_dict, mirror_mask)[0]:.2f}'
        mirror_masked_aacid = np.ma.masked_where(mirror_mask == 0, f_aacid_map)
        contra_roi_mean_pixels_aacid = f'{np.mean(mirror_masked_aacid):.2f} ± {np.std(mirror_masked_aacid):.2f}'
        
        # contra final masks
        mirror_mask_cropped_ph = (mirror_mask*ph_map)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
        mirror_mask_cropped_aacid = (mirror_mask*f_aacid_map)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w]
        if np.any(mirror_mask):
            mirror_cntr = contour_finder(mirror_mask_cropped_ph)

        # Create a scatter plot for the contour using Plotly Express
        for i, color in zip([1,2,3,4,5], [['red', 'FLORALWHITE'], ['black', 'FLORALWHITE'], ['red', 'FLORALWHITE'], ['red', 'black'], ['red', 'black']]):
            if i==1:
                a = resratio
            else:
                a = 1
            fig.add_trace(go.Scatter(x=tumor_cntr[:, 1]*a, y=tumor_cntr[:, 0]*a, line=dict(color=color[0], width=1.8)), row=row_i, col=i)
            fig.add_trace(go.Scatter(x=tumor_cntr[:, 1]*a, y=tumor_cntr[:, 0]*a, line=dict(color=color[0], width=1.8)), row=row_i, col=i)
            if np.any(mirror_mask):
                fig.add_trace(go.Scatter(x=mirror_cntr[:, 1]*a, y=mirror_cntr[:, 0]*a, line=dict(color=color[1], width=1.8)), row=row_i, col=i)
                fig.add_trace(go.Scatter(x=mirror_cntr[:, 1]*a, y=mirror_cntr[:, 0]*a, line=dict(color=color[1], width=1.8)), row=row_i, col=i)

    # Add heatmaps for the three arrays
    heatmap_aacid_brain = go.Heatmap(z=masked_aacid[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w], coloraxis='coloraxis5')
    heatmap_ph_brain = go.Heatmap(z=masked_ph[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w], coloraxis='coloraxis')
    
    fig.add_trace(heatmap_aacid_brain, row=row_i, col=4)
    fig.add_trace(heatmap_ph_brain, row=row_i, col=5)
    
    brain_roi_mean_pixels_ph = f'{np.mean(masked_ph):.2f} ± {np.std(masked_ph):.2f}'
    brain_roi_mean_z_ph= f'{aacid_roi(subject_dict, z_dict, mask)[1]:.2f}'
    brain_roi_mean_pixels_aacid = f'{np.mean(masked_aacid):.2f} ± {np.std(masked_aacid):.2f}'
    brain_roi_mean_z_aacid= f'{aacid_roi(subject_dict, z_dict, mask)[0]:.2f}'

    # highres_img_idx = 4
    t1 = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1',
                                   image_file=4)
    t2 = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')
    highres_fn, _, bruker_dataset_highres = bruker_dataset_creator(glu_mouse_fn, txt_file_name, 'T2_MSME_high')
    t2_highres = dicom_data_arranger(bruker_dataset_highres, highres_fn).astype(np.float)[highres_img_idx]

    # Loading the separately acquired water_t1t2-maps
    shift_up, shift_right = subject_dict['t_shift']
    t1 = np.roll(t1, shift=(shift_up, shift_right), axis=(0, 1))
    t2 = np.roll(t2, shift=(shift_up, shift_right), axis=(0, 1))
    t2_highres = np.roll(t2_highres, shift=(shift_up*resratio, shift_right*resratio), axis=(0, 1))

    # Add heatmaps for the three arrays
    heatmap_t2_highres = go.Heatmap(z=t2_highres[(r_c-bb_r_h)*resratio:(r_c+bb_r_h)*resratio, (c_c-bb_c_w)*resratio:(c_c+bb_c_w)*resratio], showscale=False, colorscale=custom_hot, coloraxis='coloraxis4')
    heatmap_t1 = go.Heatmap(z=(t1*mask)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w], showscale=False, zmin=1300, zmax=2300, colorscale=custom_hot, coloraxis='coloraxis2')
    heatmap_t2 = go.Heatmap(z=(t2*mask)[r_c-bb_r_h:r_c+bb_r_h, c_c-bb_c_w:c_c+bb_c_w], showscale=False, zmin=50, zmax=110, colorscale=custom_hot, coloraxis='coloraxis3')
    fig.add_trace(heatmap_t2_highres, row=row_i, col=1)
    fig.add_trace(heatmap_t1, row=row_i, col=2)
    fig.add_trace(heatmap_t2, row=row_i, col=3)
    
    # flattened dict:
    flattened_aacid_dicts.append({
        'whole': masked_aacid.compressed(),
        'tumor': tumor_masked_aacid.compressed(),
        'contra': mirror_masked_aacid.compressed(),
    })
    
    flattened_ph_dicts.append({
        'whole': masked_ph.compressed(),
        'tumor': tumor_masked_ph.compressed(),
        'contra': mirror_masked_ph.compressed(),
    })
    
    # dataframes:
    aacid_data = {
        'whole brain roi pixels': [brain_roi_mean_pixels_aacid, brain_roi_mean_z_aacid],
        'tumor roi pixels': [tumor_roi_mean_pixels_aacid, tumor_roi_mean_z_aacid],
        'contra roi pixels': [contra_roi_mean_pixels_aacid, contra_roi_mean_z_aacid],
    }
    # Define the data
    ph_data = {
        'whole brain roi pixels': [brain_roi_mean_pixels_ph, brain_roi_mean_z_ph],
        'tumor roi pixels': [tumor_roi_mean_pixels_ph, tumor_roi_mean_z_ph],
        'contra roi pixels': [contra_roi_mean_pixels_ph, contra_roi_mean_z_ph],
    }
    
    # Define the row titles
    index = ['pixels', 'z']
    
    # Create the DataFrame
    aacid_dfs.append(pd.DataFrame(aacid_data, index=index))
    ph_dfs.append(pd.DataFrame(ph_data, index=index))
    
    fig.update_yaxes(title_text=subject_dict['sub_name'], showgrid=False, row=row_i, col=1, title_font=dict(size=18))

fig.update_layout(
    template='plotly_dark',  # Set the theme to plotly dark
    title_text=f"AACID & pH map - {date} Adult Tumor Mice",
    showlegend=False,  # Hide legend
    height=450,
    width=1000,  # Set a width based on your preference
    margin=dict(l=5, r=40, t=60, b=0),  # Adjust top and bottom margins
    title=dict(x=0.02, y=0.97)  # Adjust the title position
)

# Add individual titles and separate colorbars
for i in range(5):
    for j in range(3):
        fig.update_xaxes(row=j+1, col=i+1, showgrid=False, showticklabels=False)
        fig.update_yaxes(showgrid=False, row=j+1, col=i+1, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# colorbar = {'colorscale': custom_viridis, 'cmin': 0, 'cmax': 11}

# Configure colorbar settings
colorbar_highres = {'colorscale': plotly.colors.sequential.Greys_r}
colorbar_t1w = {'colorscale': custom_hot, 'cmin': 1200, 'cmax': 2300}
colorbar_t2w = {'colorscale': custom_aggrnyl, 'cmin': 40, 'cmax': 150}
colorbar_aacid = {'colorscale': custom_viridis, 'cmin': 1, 'cmax': 1.5}  # custom_viridis
colorbar_ph = {'colorscale': custom_viridis, 'cmin': 6, 'cmax': 9}  # custom_viridis
coloraxis1_colorbar=dict(orientation='h', x=0.083, y=-0.1, len=1/6, thickness=18)

n = 5
start = 1/(n*2)
step = (1) / n
cb_y = -0.12
fig.update_layout(
    coloraxis4=colorbar_highres,
    coloraxis2=colorbar_t1w,
    coloraxis3=colorbar_t2w,
    coloraxis5=colorbar_aacid,
    coloraxis=colorbar_ph,
    coloraxis4_colorbar=dict(orientation='h', x=start, y=cb_y, len=1/n, thickness=18),
    coloraxis2_colorbar=dict(orientation='h', x=start+step, y=cb_y, len=1/n, thickness=18),
    coloraxis3_colorbar=dict(orientation='h', x=start+2*step, y=cb_y, len=1/n, thickness=18),
    coloraxis5_colorbar=dict(orientation='h', x=start+3*step, y=cb_y, len=1/n, thickness=18),
    coloraxis_colorbar=dict(orientation='h', x=start+4*step, y=cb_y, len=1/n, thickness=18),
    coloraxis4_showscale=False
)


# fig.update_yaxes(title_text='july 8', showgrid=False, row=1, col=1, title_font=dict(size=18))
# fig.update_yaxes(title_text='july 15', showgrid=False, row=2, col=1, title_font=dict(size=18))
# fig.update_yaxes(title_text='july 29', showgrid=False, row=3, col=1, title_font=dict(size=18))
# Show the plot
fig.show()

z_fig.update_layout(
    template='plotly_dark',  # Set the theme to plotly dark
    title_text=f"z spectra - {date} Adult Tumor Mice",
    showlegend=False,  # Hide legend
    height=450,
    width=400,  # Set a width based on your preference
    margin=dict(l=5, r=40, t=60, b=0),  # Adjust top and bottom margins
    title=dict(x=0.02, y=0.97)  # Adjust the title position
)
# Add individual titles and separate colorbars
for i in range(3):
    # Add horizontal lines at y=2.75 and y=3.5 to the second subplot
    z_fig.add_trace(go.Scatter(
        x=[2.75, 2.75],  # Set x values to span the width of the subplot
        y=[0, 0.7],
        mode='lines',
        line=dict(color='green', dash='dash'),
    ), row=i+1, col=1)
    z_fig.add_trace(go.Scatter(
        x=[3.5, 3.5],  # Set x values to span the width of the subplot
        y=[0, 0.7],
        mode='lines',
        line=dict(color='green', dash='dash'),
    ), row=i+1, col=1)
    z_fig.update_xaxes(row=i+1, col=1, autorange='reversed')  # Reverse the y-axis

z_fig.show()
save_path = os.path.join(current_dir, 'images', save_name, f'subject_{subject_i+1}')
if not os.path.exists(save_path):
    os.makedirs(save_path)
pio.write_image(fig, os.path.join(save_path, 'aacid_ph_map.jpeg'))

WASSR B0 mapping took 2.197 seconds
WASSR B0 mapping took 2.435 seconds
WASSR B0 mapping took 2.221 seconds


In [253]:
aacid_dfs[0]

,whole brain roi pixels,tumor roi pixels,contra roi pixels
pixels,1.35 ± 0.21,1.32 ± 0.11,1.35 ± 0.22
z,1.33,1.32,1.33


In [254]:
aacid_dfs[1]

,whole brain roi pixels,tumor roi pixels,contra roi pixels
pixels,1.34 ± 0.19,1.30 ± 0.11,1.36 ± 0.24
z,1.32,1.29,1.34


In [255]:
aacid_dfs[2]

,whole brain roi pixels,tumor roi pixels,contra roi pixels
pixels,1.33 ± 0.23,1.28 ± 0.11,1.37 ± 0.27
z,1.30,1.28,1.34


In [256]:
ph_dfs[0]

,whole brain roi pixels,tumor roi pixels,contra roi pixels
pixels,7.40 ± 0.86,7.50 ± 0.43,7.40 ± 0.89
z,7.49,7.52,7.50


In [257]:
ph_dfs[1]

,whole brain roi pixels,tumor roi pixels,contra roi pixels
pixels,7.44 ± 0.77,7.62 ± 0.45,7.36 ± 0.98
z,7.52,7.64,7.45


In [258]:
ph_dfs[2]

,whole brain roi pixels,tumor roi pixels,contra roi pixels
pixels,7.49 ± 0.92,7.68 ± 0.43,7.35 ± 1.12
z,7.59,7.70,7.45


In [259]:
# Convert to a list of tuples for DataFrame creation
ph_list = []
for df_date in flattened_ph_dicts:
    data_list = []
    for tag, np_array in df_date.items():
        data_list.extend([(tag, value) for value in np_array])
        df = pd.DataFrame(data_list, columns=['tag', 'value'])
        ph_list.append(df)
        
aacid_list = []
for df_date in flattened_aacid_dicts:
    data_list = []
    for tag, np_array in df_date.items():
        data_list.extend([(tag, value) for value in np_array])
        df = pd.DataFrame(data_list, columns=['tag', 'value'])
        aacid_list.append(df)

In [260]:
fig = go.Figure()

fig.add_trace(go.Box(
    y=np.hstack((flattened_aacid_dicts[0]['contra'], flattened_aacid_dicts[1]['contra'], flattened_aacid_dicts[2]['contra'])),
    x=np.hstack((len(flattened_aacid_dicts[0]['contra'])*['B1'], len(flattened_aacid_dicts[1]['contra'])*['B4'],  len(flattened_aacid_dicts[2]['contra'])*['C5'])),
    name='Contra',
    marker_color='#3D9970',
))

fig.add_trace(go.Box(
    y=np.hstack((flattened_aacid_dicts[0]['tumor'], flattened_aacid_dicts[1]['tumor'], flattened_aacid_dicts[2]['tumor'])),
    x=np.hstack((len(flattened_aacid_dicts[0]['tumor'])*['B1'], len(flattened_aacid_dicts[1]['tumor'])*['B4'],  len(flattened_aacid_dicts[2]['tumor'])*['C5'])),
    name='Tumor',
    marker_color='#FF4136',
))

# # Add markers for 'contra'
# fig.add_trace(go.Scatter(
#     y=[float(aacid_dfs[0].iloc[1,1]), float(aacid_dfs[1].iloc[1,1]), float(aacid_dfs[2].iloc[1,1])],
#     x=['B1', 'B4', 'C5'],
#     mode='markers',
#     name='Contra roi z',
#     marker=dict(color='#3D9970', size=6, symbol='circle')
# ))
# 
# # Add markers for 'tumor'
# fig.add_trace(go.Scatter(
#     y=[float(aacid_dfs[0].iloc[1,2]), float(aacid_dfs[1].iloc[1,2]), float(aacid_dfs[2].iloc[1,2])],
#     x=['B1', 'B4', 'C5'],
#     mode='markers',
#     name='Tumor roi z',
#     marker=dict(color='#FF4136', size=6, symbol='x')
# ))

fig.update_layout(
    yaxis_title='AACID',
    boxmode='group', # group together boxes of the different traces for each value of x,
    yaxis=dict(range=[0.5, 2.5]),  # Set y-axis limits
    width=450,
    height=300,
    margin=dict(l=20, r=40, t=60, b=10),  # Adjust top and bottom margins
    title=f'AACID pixel values - {date} Adult Tumor Mice'
)
fig.show()
pio.write_image(fig, os.path.join(save_path, 'aacid_boxplot.jpeg'))

In [261]:
import plotly.graph_objects as go

fig = go.Figure()

# fig.add_trace(go.Box(
#     y=np.hstack((flattened_ph_dicts[0]['whole'], flattened_ph_dicts[1]['whole'])),
#     x=np.hstack((len(flattened_ph_dicts[0]['whole'])*['july 8'], len(flattened_ph_dicts[1]['whole'])*['july 15'])),
#     name='whole',
#     marker_color='#FF851B',
# ))

fig.add_trace(go.Box(
    y=np.hstack((flattened_ph_dicts[0]['contra'], flattened_ph_dicts[1]['contra'], flattened_ph_dicts[2]['contra'])),
    x=np.hstack((len(flattened_ph_dicts[0]['contra'])*['B1'], len(flattened_ph_dicts[1]['contra'])*['B4'],  len(flattened_ph_dicts[2]['contra'])*['C5'])),
    name='contra',
    marker_color='#3D9970',
))

fig.add_trace(go.Box(
    y=np.hstack((flattened_ph_dicts[0]['tumor'], flattened_ph_dicts[1]['tumor'], flattened_ph_dicts[2]['tumor'])),
    x=np.hstack((len(flattened_ph_dicts[0]['tumor'])*['B1'], len(flattened_ph_dicts[1]['tumor'])*['B4'], len(flattened_ph_dicts[2]['tumor'])*['C5'])),
    name='tumor',
    marker_color='#FF4136',
))

fig.update_layout(
    yaxis_title='pH',
    boxmode='group', # group together boxes of the different traces for each value of x,
    yaxis=dict(range=[6, 9]),  # Set y-axis limits
    width=450,
    height=300,
    margin=dict(l=20, r=40, t=60, b=10),  # Adjust top and bottom margins
    title=f'pH pixel values - {date} Adult Tumor Mice'
)
fig.show()

# Save to PNG file (requires kaleido)
pio.write_image(fig, os.path.join(save_path, 'ph_boxplot.jpeg'))